In [1]:
import sys
sys.path.append('../Common/')

import CommonYFinance, CommonBinance, CommonMT5, CommonMongoDBConnector, CommonBacktest

In [2]:
# Chiến thuật EU : MA MACD STOCHASTIC EMA34 89 

def detectSignal(symbol, from_date, to_date, timeframe):
    import pandas as pd
    import plotly.graph_objects as go
    import redis
    import numpy as np
    from datetime import datetime

    # ##############################################Step 1: Lấy dữ liệu##############################################
    data = CommonMT5.CommonMT5.loaddataMT5_FromTo(symbol, from_date, to_date, timeframe)
            
    # ##############################################Step 2: Lấy dữ liệu##############################################  
    # Tính toán chọn vùng giao dịch trên khung 1H 
        # Tính EMA 34 & 89 trên khung 1H
    data['EMA34_1h'] = data['Close'].ewm(span=136, adjust = False).mean()
    data['EMA89_1h'] = data['Close'].ewm(span=356, adjust = False).mean()
        # Tính MACD
            # EMA ngắn hạn
    data['short_ema_1h'] = data['Close'].ewm(span=48, adjust=False).mean()
            # EMA dài hạn
    data['long_ema_1h']  = data['Close'].ewm(span=104, adjust=False).mean()
            # Tính MACD
    data['MACD_1h'] = data['short_ema_1h']  - data['long_ema_1h']
            # Tính Signal Line
    data['Signal_Line_1h'] = data['MACD_1h'].ewm(span=36, adjust=False).mean()
            # Vùng mua bán trên 1H
    
    # Tính toán vùng giao dịch chính khung 15 phút
    # window = 20
    atr_period = 14
    # Tính toán Stochastic 
        # Define periods
    lookback = 13
    k_period = 8
    d_period = 5
        # Adds a "n_high" column with max value of previous 14 periods
    data['n_high'] = data['High'].rolling(lookback).max()
        # Adds an "n_low" column with min value of previous 14 periods
    data['n_low'] = data['Low'].rolling(lookback).min()
        # Uses the min/max values to calculate the %k (as a percentage)
    data['%K'] = ((data['Close'] - data['n_low']) * 100 / (data['n_high'] - data['n_low'])).rolling(k_period).mean()
        # Uses the %k to calculates a SMA over the past 3 values of %k
    data['%D'] = data['%K'].rolling(d_period).mean()

    # Tính toán MACD
        # EMA ngắn hạn
    data['short_ema'] = data['Close'].ewm(span=12, adjust=False).mean()
        # EMA dài hạn
    data['long_ema']  = data['Close'].ewm(span=26, adjust=False).mean()
        # Tính MACD
    data['MACD'] = data['short_ema']  - data['long_ema']
        # Tính Signal Line
    data['Signal_Line'] = data['MACD'].ewm(span=9, adjust=False).mean()
    # Định nghĩa đường Crossing point
    def find_crossing_points(data):
        crossing_points = []
        for i in range(1, len(data)):
            if (data['%K'][i] > data['%D'][i]) and (data['%K'][i-1] < data['%D'][i-1]):
                crossing_points.append((data.index[i], 'up'))
            elif (data['%K'][i] < data['%D'][i]) and (data['%K'][i-1] > data['%D'][i-1]):
                crossing_points.append((data.index[i], 'down'))
        return crossing_points
    crossing_points = find_crossing_points(data)
    
    # Tính True Range (TR)
    data['prev_close'] = data['Close'].shift(1)  # Lấy giá đóng cửa của ngày trước đó
    data['high_low'] = data['High'] - data['Low']
    data['high_close'] = abs(data['High'] - data['prev_close'])
    data['low_close'] = abs(data['Low'] - data['prev_close'])
    data['TR'] = data[['high_low', 'high_close', 'low_close']].max(axis=1)
    # Tính ATR
    data['ATR'] = data['TR'].rolling(window=atr_period).mean()
    # TẠO TÍN HIỆU MUA BÁN
    # data['Buy_Signal'] =  crossing_points and data['MACD'] > 0 and data['Signal_Line'] > 0 and (data['EMA34_1h'] > data['EMA89_1h']) and (data['MACD_1h'] > data['Signal_Line_1h'])
    # data['Sell_Signal'] = crossing_points and data['MACD'] < 0 and data['Signal_Line'] < 0 and (data['EMA34_1h'] < data['EMA89_1h']) and (data['MACD_1h'] < data['Signal_Line_1h'])
    for i in range(len(data)):
        if (i, 'up') in crossing_points and data['MACD'][i] > 0 and data['Signal_Line'][i] > 0 and (data['EMA34_1h'][i] > data['EMA89_1h'][i]) and (data['MACD_1h'][i] > data['Signal_Line_1h'][i]):
            data.at[i, 'Buy_Signal'] = True
        elif (i, 'down') in crossing_points and data['MACD'][i] < 0 and data['Signal_Line'][i] < 0 and (data['EMA34_1h'][i] < data['EMA89_1h'][i]) and (data['MACD_1h'][i] < data['Signal_Line_1h'][i]):
            data.at[i, 'Sell_Signal'] = True
        else:
            data.at[i, 'Buy_Signal'] = False
            data.at[i, 'Sell_Signal'] = False

    print(data)
    
    # ##############################################Step 3: Đẩy dữ liệu qua Redis##############################################
    # Nếu có tín hiệu thì đẩy qua Redis
    # Datetime  Open    High    Low	Close   Volume  SMA short_ema   long_ema    MACD    Signal_Line Buy_Signal  Sell_Signal
    # Tạo kết nối Redis
    r = redis.Redis(host='localhost', port=6379, db=1, password = None)
    # Tạo hash key
    hash_key = 'OG_FX_MACD,MA'
    last_record = data.iloc[-1]
   
    # Chuyển đổi record cuối cùng thành từ điển và lưu vào Redis dưới dạng hash
    if(last_record['Buy_Signal'] == True or last_record['Sell_Signal'] == True): 
        for field, value in last_record.to_dict().items():
            # Chuyển đổi giá trị uint64 và Timestamp thành chuỗi
            if isinstance(value, pd.Timestamp):
                value = value.isoformat()
            elif isinstance(value, (int, np.uint64)):
                value = str(value)
            r.hset(hash_key, field, value)  
            r.hset(hash_key, 'Symbol', symbol)
            r.hset(hash_key, 'Insertdate', datetime.now().isoformat())
        print(last_record)   
    else:
        print('Không có tín hiệu!')
    # ##############################################Step 4: Vẽ biểu đồ##############################################  
    # data.set_index('Datetime', inplace=True)
    # # Tạo figure
    # fig = go.Figure()
    # # Biểu đồ giá đóng cửa và SMA
    # fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Giá Đóng Cửa', line=dict(color='blue')))
    # fig.add_trace(go.Scatter(x=data.index, y=data['SMA'], mode='lines', name='SMA', line=dict(color='orange')))
    # # Vẽ các điểm cho tín hiệu mua
    # buy_signals = data[data['Buy_Signal']]
    # fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', name='Buy Signal', marker=dict(symbol='triangle-up', size=10, color='green')))
    # # Vẽ các điểm cho tín hiệu bán
    # sell_signals = data[data['Sell_Signal']]
    # fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', name='Sell Signal', marker=dict(symbol='triangle-down', size=10, color='red')))
    # fig.update_layout(title='Giá Đóng Cửa và SMA', yaxis_title='Giá', xaxis_rangeslider_visible=False, height=500)
    # # Hiển thị biểu đồ
    # fig.show()

    # ####################################################################################################
    # # Tạo figure
    # fig = go.Figure()
    # # Biểu đồ MACD và Signal Line
    # fig.add_trace(go.Scatter(x=data.index, y=data['MACD'], mode='lines', name='MACD', line=dict(color='blue')))
    # fig.add_trace(go.Scatter(x=data.index, y=data['Signal_Line'], mode='lines', name='Signal Line', line=dict(color='orange')))
    # fig.update_layout(title='MACD và Signal Line', yaxis_title='MACD', xaxis_rangeslider_visible=False, height=500)
    # # Hiển thị biểu đồ
    # fig.show()

In [3]:
# from datetime import datetime, timedelta
# import schedule
# import time

# def schedule_detectSignal():

#     symbol = 'EURUSD'
#     from_date = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-%d')  # Lấy ngày hôm qua
#     to_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
#     timeframe = 15 # 1m
#     detectSignal(symbol, from_date, to_date, timeframe)

# # Lên lịch hàm scan_market để chạy mỗi 15 phút
# schedule.every(15).minutes.do(schedule_detectSignal)

# while True:
#     schedule.run_pending()  # Hàm này được gọi liên tục để kiểm tra xem có công việc nào đã đến thời gian cần thực hiện hay không
#     time.sleep(10)

In [4]:
from datetime import datetime, timedelta
import schedule
import time

def schedule_detectSignal():
    symbol = 'EURUSD'
    from_date = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-%d')  # Lấy ngày hôm qua
    to_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
    timeframe = 30 # 1m
    detectSignal(symbol, from_date, to_date, timeframe)

# Danh sách các phút cụ thể bạn muốn hàm được chạy
run_minutes = [16,39,46,1]

while True:
    current_time = datetime.now()
    current_minute = current_time.minute
    # Kiểm tra xem phút hiện tại có nằm trong danh sách các phút cần chạy hàm không
    if current_minute in run_minutes:
        # Kiểm tra xem đã chạy hàm trong phút hiện tại hay chưa
        last_run = getattr(schedule_detectSignal, 'last_run', None)
        if last_run is None or last_run != current_minute:
            schedule_detectSignal()
            # Lưu lại phút cuối cùng mà hàm đã chạy
            setattr(schedule_detectSignal, 'last_run', current_minute)
    # Nghỉ 10 giây trước khi kiểm tra lại
    time.sleep(10)


C:\Users\PC-DELL-CU\AppData\Local\Temp\ipykernel_7980\1670491362.py:83: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.at[i, 'Buy_Signal'] = False
C:\Users\PC-DELL-CU\AppData\Local\Temp\ipykernel_7980\1670491362.py:84: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.at[i, 'Sell_Signal'] = False


               Datetime     Open     High      Low    Close  Volume  EMA34_1h  \
0   2024-01-09 17:00:00  1.09449  1.09458  1.09343  1.09362    3120  1.093620   
1   2024-01-09 17:30:00  1.09362  1.09364  1.09166  1.09171    3283  1.093592   
2   2024-01-09 18:00:00  1.09170  1.09272  1.09096  1.09253    2914  1.093577   
3   2024-01-09 18:30:00  1.09252  1.09343  1.09218  1.09335    2335  1.093573   
4   2024-01-09 19:00:00  1.09336  1.09381  1.09293  1.09369    2353  1.093575   
..                  ...      ...      ...      ...      ...     ...       ...   
117 2024-01-12 03:30:00  1.09753  1.09834  1.09742  1.09819     889  1.095715   
118 2024-01-12 04:00:00  1.09818  1.09845  1.09794  1.09805    1176  1.095750   
119 2024-01-12 04:30:00  1.09805  1.09814  1.09774  1.09787     549  1.095781   
120 2024-01-12 05:00:00  1.09787  1.09818  1.09786  1.09810     464  1.095814   
121 2024-01-12 05:30:00  1.09807  1.09819  1.09767  1.09772     215  1.095842   

     EMA89_1h  short_ema_1h